# data loader

In [8]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data\\Blue Light Blue Color Blocks Flight Attendant CV.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()
print(data)


[Document(metadata={'source': 'data\\Blue Light Blue Color Blocks Flight Attendant CV.pdf', 'page': 0, 'page_label': '1'}, page_content='AMINA IQBAL \nML ENGINEER \n+923232983818 amminaiqball@gmail.com\nPORTFOLIO\nTHANKS A LOT\nPROFILE\nA passionate Machine Learning intern with hands-on experience in\nbuilding ML models and developing interactive applications using\nStreamlit and LangChain. Successfully created chatbots and integrated\nML models into Streamlit applications. Currently working on a Retrieval-\nAugmented Generation (RAG) system, leveraging advanced AI\ntechniques for efficient data retrieval and generation. Eager to apply and\nexpand my knowledge in AI and ML through innovative projects.\nMSc in Math & Physics\nUniversity of Punjab\n2019 - 2023\nGovt. Degree College for Women\nIntermidiate in Engeenring\n2016 - 2018\nEDUCATION\n SKILLS\nEnglish\nUrdu\nLANGUAGE\nMachine Learning\nRetrieval Augmented\nGeneration\nDeep Learning (Basic)\nhttps://www.linkedin.com/in/amminaiqba

# data splitter

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(data)
# len(texts)
print(texts[0])

page_content='AMINA IQBAL 
ML ENGINEER 
+923232983818 amminaiqball@gmail.com
PORTFOLIO
THANKS A LOT
PROFILE' metadata={'source': 'Blue Light Blue Color Blocks Flight Attendant CV.pdf', 'page': 0, 'page_label': '1'}


# embedings

In [ ]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual OpenAI API key
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# embeddings = embed_model.embed_documents(str(texts))

# vector db search

In [33]:

from Pinecone_class import pincoine_ob
import os



In [34]:
pincoine_ob.check_index("cvdata-new")

'Your index name cvdata-new Found'

In [35]:
print(type(texts))
# pdf_data = str(texts)
# print(type(pdf_data))
# print(pdf_data)

pincoine_ob.check_index(index='cvdata-new')

<class 'list'>


'Your index name cvdata-new Found'

In [36]:
res = pincoine_ob.insert_data_in_index(documents=texts,embeddings=embeddings,index_name='cvdata-new')
print(res)

Your Data insert in cvdata-new successfully
None


In [37]:
pincoine_ob.insert_data_in_namespace(documents= texts,embeddings = embeddings,index_name = "cvdata-new",name_space = "data")

Your Name space data is Created successfully


In [38]:
# pincoine_ob.insert_data_in_index(embeddings= embeddings,documents = texts,index_name = "cvdata")

result = pincoine_ob.insert_data_in_index(documents=texts,embeddings=embeddings,index_name='cvdata-new')

print(result)

Your Data insert in cvdata-new successfully
None


In [39]:
vectordb = pincoine_ob.retrieve_from_namespace(index_name = "cvdata",embeddings = embeddings, name_space = "data")


In [40]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [48]:
retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
docs = retriever.invoke("what student can do")
print(docs[0].page_content)

EXPERIENCE
Completed Python programming assignments in basic


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("what is student contact information")

'Student contact information is:\nPhone number: +923232983818\nEmail: amminaiqball@gmail.com\nFacebook profile: https://www.facebook.com/amminaiqball'

# LLM Chain

👉 LLM ko direct prompt bhejne aur response lene ke liye

✔ Use case: Jab sirf LLM ka response chahiye without retrieval.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatOpenAI()

chain = prompt | llm
print(chain.invoke({"topic": "AI"}))


# Retrieval Chain (Simple RAG)

👉 LLM ko documents se relevant information retrieve kar ke response generate karwana

 Use case: Jab documents se relevant information extract karni ho.

In [ ]:
retriever = vectorstore.as_retriever()

chain = retriever | llm
print(chain.invoke("What is LangChain?"))


# 🔹 3. Stuff Retrieval Chain

👉 Multiple retrieved documents ko ek jagah combine kar ke use karna

✔ Use case: Jab multiple documents ko combine kar ke response generate karna ho.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_documents_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
chain = retriever | combine_documents_chain
print(chain.invoke("What is LangChain?"))


# 🔹 4. Map-Reduce Retrieval Chain


👉 Agar documents bohot zyada hain to unko pehle summarize karna aur phir final answer dena

✔ Use case: Large document sets ke saath kaam karne ke liye.

In [ ]:
from langchain.chains.combine_documents import create_map_reduce_documents_chain

combine_documents_chain = create_map_reduce_documents_chain(llm)
chain = retriever | combine_documents_chain
print(chain.invoke("Explain deep learning?"))


# 🔹 5. Refine Retrieval Chain

👉 Pehle ek initial answer generate karna, phir usko refine karna

✔ Use case: Jab incrementally answer improve karna ho.

In [ ]:
from langchain.chains.combine_documents import create_refine_documents_chain

combine_documents_chain = create_refine_documents_chain(llm)
chain = retriever | combine_documents_chain
print(chain.invoke("Tell me about OpenAI?"))


# 🔹 6. Conversational Retrieval Chain (Memory-Based RAG)

👉 User ki pichli queries ka context yaad rakhna

✔ Use case: Jab chatbot ko user ka pichla conversation yaad rakhna ho.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)
print(chain.invoke({"question": "Tell me about Tesla?"}))


# 🔹 7. Runnable RAG Chain (Pipelined Execution)

👉 LangChain ki new "Runnable API" ka use karna

✔ Use case: Jab custom modular pipeline chahiye jo scalable aur flexible ho.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(chain.invoke("What is AI?"))


# 🔹 8. Multi-Retrieval Chain

👉 Agar multiple retrievers (like Pinecone + FAISS) use karne ho

✔ Use case: Jab multiple knowledge sources ko combine karna ho.

In [ ]:
from langchain.chains import MultiRetrievalQAChain

retriever1 = vectorstore1.as_retriever()
retriever2 = vectorstore2.as_retriever()

chain = MultiRetrievalQAChain.from_retrievers(llm, retrievers=[retriever1, retriever2])
print(chain.invoke({"query": "Explain blockchain"}))
